In [1]:
# Libraries

from IPython.display import display
from pandas          import DataFrame

import itertools
import numpy
import os
import pandas
import sys

In [2]:
# Root directory

ROOT = os.getcwd()

while not ROOT.endswith('upolanc-thesis') :
	ROOT = os.path.abspath(os.path.join(ROOT, os.pardir))

	if len(ROOT) < len('upolanc-thesis') :
		if   platform.system() == 'Linux' :
			ROOT = '/d/hpc/home/up4472/workspace/upolanc-thesis'
		elif platform.system() == 'Windows' :
			ROOT = 'C:\\Developer\\Workspace\\PyCharm\\Projects\\upolanc-thesis'
		else :
			raise ValueError()

		print(f'Warning : could not find correct directory, using default : {ROOT}')
		break

if ROOT not in sys.path :
	sys.path.append(ROOT)

os.chdir(ROOT)

In [3]:
# Code

from source.python.io import loader as data_loader
from source.python.io import writer as data_writer

In [4]:
# List temporary folder content

ROOT_DIR = os.path.join(ROOT, 'reports')

for path in os.listdir(ROOT_DIR) :
	if path.startswith('model') or path.startswith('tuner') :
		print(os.path.join(ROOT_DIR, path))

C:\Developer\Workspace\PyCharm\Projects\upolanc-thesis\reports\model-w-gene-0500-global-mean
C:\Developer\Workspace\PyCharm\Projects\upolanc-thesis\reports\model-w-gene-0500-tissue-mean
C:\Developer\Workspace\PyCharm\Projects\upolanc-thesis\reports\model-w-gene-0500-tissue-mean-explode
C:\Developer\Workspace\PyCharm\Projects\upolanc-thesis\reports\model-w-gene-0500-tissue-mean-seedling
C:\Developer\Workspace\PyCharm\Projects\upolanc-thesis\reports\model-z-gene-0500-global-mean
C:\Developer\Workspace\PyCharm\Projects\upolanc-thesis\reports\model-z-gene-0500-tissue-mean
C:\Developer\Workspace\PyCharm\Projects\upolanc-thesis\reports\model-z-gene-0500-tissue-mean-explode
C:\Developer\Workspace\PyCharm\Projects\upolanc-thesis\reports\model-z-gene-0500-tissue-mean-seedling
C:\Developer\Workspace\PyCharm\Projects\upolanc-thesis\reports\tuner-w-gene-0250-25-tissue-mean-explode
C:\Developer\Workspace\PyCharm\Projects\upolanc-thesis\reports\tuner-w-gene-0500-25-global-mean
C:\Developer\Workspace

In [5]:
# Define constants

MODELS        = ['zrimec2020r', 'washburn2019r']
SEQUENCES     = ['transcript', 'gene']
MODEL_EPOCHS  = [100, 250, 500, 1000]
TUNE_EPOCHS   = [25, 50]
TUNE_TRIALS   = [100, 250, 500, 1000]
TARGETS0      = ['tissue', 'group', 'global']
TARGETS1      = ['mean', 'max']
TARGETS2      = ['all', 'explode', 'seedling', None]

# 1. Helpers

In [6]:
# Define helpers

def flatten_dict (data, pk, sep) :
	"""
	Doc
	"""

	items = []

	for k, v in data.items() :
		if pk :
			nk = pk + sep + k
		else :
			nk = k

		if isinstance(v, dict) :
			items.extend(flatten_dict(v, nk, sep).items())
		else :
			items.append((nk, v))

	return dict(items)

In [7]:
# Define helpers

def recover_dataframe (logdir) :
	"""
	Doc
	"""

	data = dict()

	for log in os.scandir(logdir) :
		if not os.path.isdir(log.path) :
			continue

		params = os.path.join(log.path, 'params.json')
		result = os.path.join(log.path, 'result.json')

		with open(result, mode = 'r') as handle :
			result = handle.readlines()[-1]
			result = json.loads(result)
			result = flatten_dict(result, None, '/')

		data[log.name] = result

	dataframe = DataFrame.from_dict(data, orient = 'index')
	dataframe = dataframe.reset_index(names = ['logdir'])
	dataframe = dataframe.replace('null', numpy.nan)

	return dataframe

In [8]:
# Define helpers

def dataframe_tuner_format (dataframe) :
	"""
	Doc
	"""

	columns = []
	custom  = [
		'Model', 'Sequence', 'Trials', 'Epochs',
		'Target0', 'Target1', 'Target2'
	]

	for column in custom :
		if column in dataframe.columns :
			columns.append(column)

	columns.extend([
		'logdir',
		'valid_loss', 'valid_mae', 'valid_r2',
		'train_loss',
		'training_iteration',
		'config/optimizer/name', 'config/optimizer/lr', 'config/optimizer/momentum', 'config/optimizer/decay',
		'config/scheduler/name', 'config/dataset/batch_size', 'config/model/dropout'
	])

	dataframe = dataframe[columns].copy()

	dataframe = dataframe.rename(columns = {
		'logdir'                    : 'ID',
		'valid_loss'                : 'Valid_MSE',
		'valid_mae'                 : 'Valid_MAE',
		'valid_r2'                  : 'Valid_R2',
		'train_loss'                : 'Train_MSE',
		'training_iteration'        : 'Epoch',
		'config/optimizer/name'     : 'Optimizer',
		'config/optimizer/lr'       : 'Learning_Rate',
		'config/optimizer/momentum' : 'Momentum',
		'config/optimizer/decay'    : 'Decay',
		'config/scheduler/name'     : 'Scheduler',
		'config/dataset/batch_size' : 'Batch_Size',
		'config/model/dropout'      : 'Dropout'
	})

	dataframe = dataframe.astype({
		'ID'            : str,
		'Valid_MSE'     : float,
		'Valid_MAE'     : float,
		'Valid_R2'      : float,
		'Train_MSE'     : float,
		'Epoch'         : int,
		'Optimizer'     : str,
		'Learning_Rate' : float,
		'Momentum'      : float,
		'Decay'         : float,
		'Scheduler'     : str,
		'Batch_Size'    : int,
		'Dropout'       : float
	})

	dataframe['Valid_MSE'    ] = dataframe['Valid_MSE'    ].map('{:.9f}'.format)
	dataframe['Valid_MAE'    ] = dataframe['Valid_MAE'    ].map('{:.9f}'.format)
	dataframe['Valid_R2'     ] = dataframe['Valid_R2'     ].map('{:.9f}'.format)
	dataframe['Train_MSE'    ] = dataframe['Train_MSE'    ].map('{:.9f}'.format)
	dataframe['Learning_Rate'] = dataframe['Learning_Rate'].map('{:.9f}'.format)
	dataframe['Momentum'     ] = dataframe['Momentum'     ].map('{:.9f}'.format)
	dataframe['Decay'        ] = dataframe['Decay'        ].map('{:.9f}'.format)
	dataframe['Dropout'      ] = dataframe['Dropout'      ].map('{:.3f}'.format)

	return dataframe

# 2. Tuner Reports

In [9]:
# Cache

tuner_reports = dict()

In [10]:
# Display top few results for tuners

for item in itertools.product(MODELS, SEQUENCES, TUNE_TRIALS, TUNE_EPOCHS, TARGETS0, TARGETS1, TARGETS2) :
	model   = item[0]
	data    = item[1]
	trial   = item[2]
	epoch   = item[3]
	target0 = item[4]
	target1 = item[5]
	target2 = item[6]

	if target2 == None :
		key = 'tuner-{}-{}-{:04d}-{}-{}-{}'.format(model[0], data, trial, epoch, target0, target1)
	else :
		key = 'tuner-{}-{}-{:04d}-{}-{}-{}-{}'.format(model[0], data, trial, epoch, target0, target1, target2)

	folder   = os.path.join(ROOT_DIR, key)

	if not os.path.exists(folder) :
		continue

	filename = os.path.join(folder, 'report.csv')
	logdir   = os.path.join(folder, 'raytune')

	if not os.path.exists(filename) :
		dataframe = recover_dataframe(logdir)
		dataframe.to_csv(filename)
	else :
		dataframe = pandas.read_csv(filename, index_col = [0])
		dataframe = dataframe.sort_values('valid_loss', ascending = True)
		dataframe = dataframe.reset_index()

	dataframe['logdir'] = dataframe['logdir'].str.split('/')
	dataframe['logdir'] = dataframe['logdir'].str[-1]

	print(filename)
	display(dataframe_tuner_format(dataframe).head(n = 5))
	print()

	tuner_reports[key] = dataframe

C:\Developer\Workspace\PyCharm\Projects\upolanc-thesis\reports\tuner-z-gene-0500-25-tissue-mean-explode\report.csv


,ID,Valid_MSE,Valid_MAE,Valid_R2,Train_MSE,Epoch,Optimizer,Learning_Rate,Momentum,Decay,Scheduler,Batch_Size,Dropout
0,20eeb340,0.684452041,0.676233979,0.236752570,0.592882374,25,adam,0.002791531,0.982148616,0.002005991,linear,32,0.088
1,7f9fa9f0,0.699082444,0.678264028,0.250923365,0.591841617,25,adam,0.001321251,0.689600467,0.021878420,linear,128,0.162
2,928c1555,0.706974659,0.687647030,0.234133556,0.461094682,25,adam,0.000246633,0.971163729,0.095042507,linear,64,0.195
3,55be1e54,0.716455332,0.698645344,0.199709773,0.652678863,25,adam,0.001281808,0.837902733,0.019473303,linear,32,0.051
4,efd5e28a,0.721549600,0.696148930,0.217384547,0.614779452,25,adam,0.000855270,0.830793560,0.049112288,linear,64,0.090



C:\Developer\Workspace\PyCharm\Projects\upolanc-thesis\reports\tuner-z-gene-1000-25-tissue-mean\report.csv


,ID,Valid_MSE,Valid_MAE,Valid_R2,Train_MSE,Epoch,Optimizer,Learning_Rate,Momentum,Decay,Scheduler,Batch_Size,Dropout
0,cf6fb0e1,0.735560574,0.705544795,0.200882790,0.651009062,25,adam,0.000855270,0.830793560,0.049112288,linear,64,0.090
1,1846f4ce,0.764939556,0.720136632,0.164727750,0.658987201,25,adam,0.000775059,0.546805620,0.062851040,linear,64,0.067
2,ef732097,0.765931757,0.711856788,0.146628990,0.721336816,25,adam,0.003465635,0.586310043,0.008517381,linear,32,0.023
3,3352ee42,0.766049114,0.715656379,0.149813600,0.623199575,25,adam,0.000622635,0.505144153,0.047928093,plateau,32,0.058
4,e30e5bd7,0.771964207,0.722303475,0.164113480,0.767572030,25,adam,0.002342349,0.638099584,0.051881685,linear,64,0.017



C:\Developer\Workspace\PyCharm\Projects\upolanc-thesis\reports\tuner-z-gene-1000-25-global-mean\report.csv


...

,ID,Valid_MSE,Valid_MAE,Valid_R2,Train_MSE,Epoch,Optimizer,Learning_Rate,Momentum,Decay,Scheduler,Batch_Size,Dropout
0,e3290128,0.621668590,0.636150431,0.315615270,0.591602998,25,adam,0.000267025,0.692889278,0.005743363,step,32,0.474
1,c07e6432,0.635280578,0.639042504,0.319446060,0.621541332,25,adam,0.001025030,0.863352885,0.002270048,linear,64,0.423
2,3444ffaa,0.636669345,0.632291760,0.298259400,0.561920608,25,adam,0.000176379,0.635520648,0.006127512,step,32,0.451
3,b84eacdf,0.639444885,0.641929290,0.295779650,0.516083216,25,adam,0.000108045,0.518853216,0.004795015,exponential,32,0.426
4,78056277,0.640462652,0.646407102,0.313010570,0.599876152,25,adam,0.000526629,0.622410583,0.007077184,exponential,64,0.394



C:\Developer\Workspace\PyCharm\Projects\upolanc-thesis\reports\tuner-w-gene-0500-25-tissue-mean\report.csv


,ID,Valid_MSE,Valid_MAE,Valid_R2,Train_MSE,Epoch,Optimizer,Learning_Rate,Momentum,Decay,Scheduler,Batch_Size,Dropout
0,e4f628ef,0.631963488,0.637176341,0.291523460,0.588952191,25,adam,0.000394382,0.668462266,0.000201816,linear,32,0.400
1,25f89ddd,0.637038063,0.645835373,0.283981920,0.607634138,25,adam,0.000701963,0.709807940,0.002135057,exponential,32,0.475
2,16117fc3,0.643157018,0.644499978,0.275536980,0.576568644,25,adam,0.000706512,0.709384347,0.002045684,exponential,32,0.427
3,6f3c3d1b,0.643363160,0.649148937,0.277366220,0.591596577,25,adam,0.000706821,0.683203566,0.002429887,exponential,32,0.459
4,b2acd9b3,0.646323256,0.648424289,0.278969020,0.647220928,25,adam,0.001134060,0.756605390,0.004585922,exponential,32,0.393



C:\Developer\Workspace\PyCharm\Projects\upolanc-thesis\reports\tuner-w-gene-0500-25-global-mean\report.csv


,ID,Valid_MSE,Valid_MAE,Valid_R2,Train_MSE,Epoch,Optimizer,Learning_Rate,Momentum,Decay,Scheduler,Batch_Size,Dropout
0,c4f47f56,0.563137508,0.605040411,0.296825740,0.558655353,25,adam,0.001815871,0.661281875,0.000395388,exponential,32,0.470
1,e1e1c21d,0.568019972,0.607647874,0.298890920,0.562816817,25,adam,0.000462248,0.515364561,0.001384209,exponential,32,0.489
2,a384e82b,0.569129729,0.604030127,0.294969980,0.562260905,25,adam,0.001528070,0.525375074,0.000393352,exponential,32,0.483
3,a988602f,0.572181058,0.612585602,0.278019700,0.522021200,25,adam,0.001308660,0.654645708,0.000412352,exponential,32,0.472
4,cb41bf73,0.583636552,0.625262818,0.286321800,0.597019201,25,adam,0.000763213,0.841775612,0.006447223,exponential,32,0.444


In [11]:
# Display overall prediction

R2_THRESHOLD = 0.00

for key, dataframe in tuner_reports.items() :
	nan_mse = dataframe['valid_loss'].isnull().values.ravel().sum()
	pos_r2  = dataframe[dataframe['valid_r2'] > R2_THRESHOLD]['valid_r2'].count()
	neg_r2  = dataframe[dataframe['valid_r2'] < R2_THRESHOLD]['valid_r2'].count()

	print('{:40s} : Size = {:4d} || NaN Loss = {:4d} || Pos R2 = {:4d} || Neg R2 = {:4d}'.format(key, len(dataframe), nan_mse, pos_r2, neg_r2))

tuner-z-gene-0500-25-tissue-mean-explode : Size =  500 || NaN Loss =   44 || Pos R2 =  233 || Neg R2 =  223
tuner-z-gene-1000-25-tissue-mean         : Size = 1000 || NaN Loss =  565 || Pos R2 =   86 || Neg R2 =  349
tuner-z-gene-1000-25-global-mean         : Size = 1000 || NaN Loss =  243 || Pos R2 =  198 || Neg R2 =  559
tuner-w-gene-0250-25-tissue-mean-explode : Size =  250 || NaN Loss =   24 || Pos R2 =  170 || Neg R2 =   56
tuner-w-gene-0500-25-tissue-mean         : Size =  500 || NaN Loss =    6 || Pos R2 =  306 || Neg R2 =  188
tuner-w-gene-0500-25-global-mean         : Size =  500 || NaN Loss =  235 || Pos R2 =  172 || Neg R2 =   93


In [12]:
data = None

for key, dataframe in tuner_reports.items() :
	keys = key.split('-')

	model    = keys[1]
	sequence = keys[2]
	trials   = keys[3]
	epochs   = keys[4]
	target0  = keys[5]
	target1  = keys[6]

	if len(keys) == 8 :
		target2 = keys[7]
	else :
		target2 = None

	if model == 'z' : model = 'zrimec2020'
	if model == 'w' : model = 'washburn2019'

	dataframe = dataframe.copy()
	dataframe.insert(0, 'Model',    model)
	dataframe.insert(1, 'Sequence', keys[2])
	dataframe.insert(4, 'Target0',  keys[5])
	dataframe.insert(5, 'Target1',  keys[6])
	dataframe.insert(5, 'Target2',  keys[7] if len(keys) == 8 else None)

	if data is None :
		data = dataframe
	else :
		data = pandas.concat((data, dataframe))

data = data.sort_values('valid_r2', ascending = False, na_position = 'last')
data = data.reset_index()

dataframe_tuner_format(data).head(n = 50)

,Model,Sequence,Target0,Target1,Target2,ID,Valid_MSE,Valid_MAE,Valid_R2,Train_MSE,Epoch,Optimizer,Learning_Rate,Momentum,Decay,Scheduler,Batch_Size,Dropout
0,washburn2019,gene,tissue,mean,explode,c07e6432,0.635280578,0.639042504,0.319446060,0.621541332,25,adam,0.001025030,0.863352885,0.002270048,linear,64,0.423
1,washburn2019,gene,tissue,mean,explode,e3290128,0.621668590,0.636150431,0.315615270,0.591602998,25,adam,0.000267025,0.692889278,0.005743363,step,32,0.474
2,washburn2019,gene,tissue,mean,explode,78056277,0.640462652,0.646407102,0.313010570,0.599876152,25,adam,0.000526629,0.622410583,0.007077184,exponential,64,0.394
3,washburn2019,gene,tissue,mean,explode,9a7471c3,0.652823501,0.646436032,0.309993330,0.647246957,25,adam,0.000734194,0.929015752,0.000168554,plateau,128,0.426
4,washburn2019,gene,tissue,mean,explode,8039f99d,0.644801771,0.649618278,0.307750500,0.596913834,25,adam,0.000123382,0.944750722,0.007468796,plateau,64,0.326
5,washburn2019,gene,tissue,mean,explode,9a2bc047,0.646856915,0.650021511,0.305930530,0.567277230,25,adam,0.000217920,0.525867902,0.009510614,exponential,64,0.398
6,washburn2019,gene,tissue,mean,explode,0b5594a1,0.649560788,0.651824468,0.304297400,0.603778491,25,adam,0.000265706,0.571394436,0.009725213,exponential,64,0.471
7,washburn2019,gene,global,mean,None,e1e1c21d,0.568019972,0.607647874,0.298890920,0.562816817,25,adam,0.000462248,0.515364561,0.001384209,exponential,32,0.489
8,washburn2019,gene,tissue,mean,explode,3444ffaa,0.636669345,0.632291760,0.298259400,0.561920608,25,adam,0.000176379,0.635520648,0.006127512,step,32,0.451
9,washburn2019,gene,global,mean,None,c4f47f56,0.563137508,0.605040411,0.296825740,0.558655353,25,adam,0.001815871,0.661281875,0.000395388,exponential,32,0.470


In [13]:
# Print best configuration for specific model tuning

# a = tuner_reports['tuner-z-gene-1000-25-global-mean'].iloc[1, :].to_dict()

# for k in sorted(list(a.keys())) :
# 	if not k.startswith('config') :
# 		continue

# 	v = a[k]

# 	if k.startswith('config') : k = k[7:]
# 	if isinstance(k, str)   : k = '"' + k + '"'
# 	if isinstance(v, str)   : v = '"' + v + '"'
# 	if isinstance(v, float) : v = '{:.18f}'.format(v)

# 	print('\t{:30} : {},'.format(k, v))

# 3. Model Reports

In [14]:
# Cache

model_reports = pandas.DataFrame(columns = [
	'Type', 'Epochs', 'Target_0', 'Target_1', 'Target_2', 'Model',
	'Optimizer', 'Learning_Rate', 'Momentum', 'Decay', 'Scheduler',
	'Batch_Size', 'Dropout', 'Epoch',
	'Valid_MSE', 'Eval_MSE', 'Eval_MAE', 'Eval_R2'
])

In [15]:
# Display top few results for tuners

for item in itertools.product(MODELS, SEQUENCES, MODEL_EPOCHS, TARGETS0, TARGETS1, TARGETS2) :
	model   = item[0]
	data    = item[1]
	epoch   = item[2]
	target0 = item[3]
	target1 = item[4]
	target2 = item[5]

	if target2 == None :
		key = 'model-{}-{}-{:04d}-{}-{}'.format(model[0], data, epoch, target0, target1)
	else :
		key = 'model-{}-{}-{:04d}-{}-{}-{}'.format(model[0], data, epoch, target0, target1, target2)

	folder   = os.path.join(ROOT_DIR, key)

	if not os.path.exists(folder) :
		continue

	config = os.path.join(folder, 'config.json')
	report = os.path.join(folder, 'report.json')

	if not os.path.exists(config) : continue
	if not os.path.exists(report) : continue

	config = data_loader.load_json(filename = config)
	report = data_loader.load_json(filename = report)

	model_reports.loc[-1] = [
		data,
		epoch,
		target0,
		target1,
		target2,
		model,
		config['optimizer/name'],
		config['optimizer/lr'],
		config['optimizer/momentum'],
		config['optimizer/decay'],
		config['scheduler/name'],
		config['dataset/batch/train'],
		config['model/dropout'],
		report['evaluation/best/epoch'],
		report['evaluation/best/loss'],
		report['evaluation/best/mse/mean'],
		report['evaluation/best/mae/mean'],
		report['evaluation/best/r2/mean']
	]

	model_reports.index = model_reports.index + 1
	model_reports       = model_reports.sort_index()

model_reports['Learning_Rate'] = model_reports['Learning_Rate'].astype(float).map('{:.9f}'.format)
model_reports['Momentum'     ] = model_reports['Momentum'     ].astype(float).map('{:.9f}'.format)
model_reports['Decay'        ] = model_reports['Decay'        ].astype(float).map('{:.9f}'.format)
model_reports['Dropout'      ] = model_reports['Dropout'      ].astype(float).map('{:.3f}'.format)

model_reports['Valid_MSE'    ] = model_reports['Valid_MSE'    ].astype(float).map('{:.9f}'.format)
model_reports['Eval_MSE'     ] = model_reports['Eval_MSE'     ].astype(float).map('{:.9f}'.format)
model_reports['Eval_MAE'     ] = model_reports['Eval_MAE'     ].astype(float).map('{:.9f}'.format)
model_reports['Eval_R2'      ] = model_reports['Eval_R2'      ].astype(float).map('{:.9f}'.format)

model_reports.sort_values('Eval_R2', ascending = False)

,Type,Epochs,Target_0,Target_1,Target_2,Model,Optimizer,Learning_Rate,Momentum,Decay,Scheduler,Batch_Size,Dropout,Epoch,Valid_MSE,Eval_MSE,Eval_MAE,Eval_R2
0,gene,500,global,mean,None,washburn2019r,adam,0.001815871,0.661281875,0.000395388,exponential,32,0.470,32,0.616498342,0.557999268,0.599265450,0.308064580
1,gene,500,tissue,mean,None,washburn2019r,adam,0.001815871,0.661281875,0.000395388,exponential,32,0.470,23,0.656342707,0.655258795,0.652996713,0.268879235
2,gene,500,tissue,mean,seedling,washburn2019r,adam,0.001815871,0.661281875,0.000395388,exponential,32,0.470,26,0.591230285,0.647375628,0.650652180,0.261054575
4,gene,500,global,mean,None,zrimec2020r,adam,0.000855270,0.830793560,0.049112288,linear,64,0.090,43,0.615248383,0.648892189,0.655135765,0.250445098
6,gene,500,tissue,mean,seedling,zrimec2020r,adam,0.000855270,0.830793560,0.049112288,linear,64,0.090,31,0.673128116,0.652908151,0.659634473,0.246836364
5,gene,500,tissue,mean,None,zrimec2020r,adam,0.000855270,0.830793560,0.049112288,linear,64,0.090,59,0.686891409,0.705667055,0.685929240,0.233080164
3,gene,500,tissue,mean,explode,washburn2019r,adam,0.001815871,0.661281875,0.000395388,exponential,32,0.470,11,0.658476066,0.709610886,0.696640393,0.215825811
7,gene,500,tissue,mean,explode,zrimec2020r,adam,0.000855270,0.830793560,0.049112288,linear,64,0.090,21,0.692038413,0.705530821,0.678654226,0.215524673
